In [59]:
import glob
import io
import numpy as np
import os
import pandas as pd
import PyPDF2
import re
import shutil
import sys
import traceback
import threading
import warnings
import win32com.client as win32

from openpyxl import load_workbook, Workbook
from openpyxl.styles import Alignment, Border, Font, NamedStyle, PatternFill, Side
from openpyxl.utils import get_column_letter, range_boundaries
from openpyxl.cell.cell import MergedCell
from pandas import Timestamp
from pathlib import Path
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from tqdm import tqdm

In [60]:
dir_Arquivos = Path("C:/Users/jaquelinerufino/OneDrive - KPMG/Documents/Desenvolvimento/Lighthouse/Itaú/01. Originais/")
dir_ContratosFinais = Path("C:/Users/jaquelinerufino/OneDrive - KPMG/Documents/Desenvolvimento/Lighthouse/Itaú/05. Output/Contratos Finais/2018/")
output_pdf = Path("C:/Anexo N_C/pdf")
output_excel = Path("C:/Prioritários/150 faltantes/Excel/")

status_cols = ["Contrato","Cliente - Status","CNPJ/CPF - Status","Valor - Status","Data do Contrato - Status","Data de Liquidação - Status",'Quantidade de Parcelas - Status']


In [61]:
itens = pd.read_csv(Path.joinpath(dir_Arquivos,"150 Contratos.txt"), dtype=str, names=['Contrato'], skiprows=1 )
comparativo = pd.read_excel(Path.joinpath(dir_Arquivos,"Status_2018.xlsx"),dtype={'Contrato':str})
caminho_base = Path.joinpath(dir_Arquivos,"Modelo_Contrato_03.xlsx")
path_base = Path.joinpath(dir_Arquivos,"DOC_BASE_Reprocessado (1).xlsx")


In [62]:

df_contratuais_vs_registro_contabil= pd.read_csv(Path.joinpath(dir_ContratosFinais,"Quadro 1 - Dados Contratuais x Registro Contábil.csv"),encoding='latin1',sep=",", dtype=str)
df_demonstracao_valor_anual= pd.read_csv(Path.joinpath(dir_ContratosFinais,"Quadro 2 - Superveniência.csv"),encoding='latin1',sep=",", dtype=str)
df_efeito_resultado_contabil = pd.read_csv(Path.joinpath(dir_ContratosFinais,"Quadro 3 - Resultado Contábil.csv"),encoding='latin1',sep=",", dtype=str)
df_base_calculo_irpj = pd.read_csv(Path.joinpath(dir_ContratosFinais,"Quadro 4 - IRPJ.csv"),encoding='latin1',sep=",", dtype=str)
df_base_calculo_csll = pd.read_csv(Path.joinpath(dir_ContratosFinais,"Quadro 5 - CSSL.csv"),encoding='latin1',sep=",", dtype=str)
df_demonstrativo_efeito_nulo = pd.read_csv(Path.joinpath(dir_ContratosFinais,"Quadro 6 - Efeito Nulo da Superveniência_Insuficiência.csv"),encoding='latin1',sep=",",dtype=str)

In [63]:
lst_itens = itens['Contrato'].tolist()

comparativo['Contrato'] = comparativo['Contrato'].astype(str)
comparativo = comparativo[comparativo["Contrato"].isin(lst_itens)]

## Definitions

### criar_modelo_excel

In [64]:
def criar_modelo_excel(
    nome_arquivo,
    cor_cabecalho1="FFFF00",
    texto_cabecalho1="Cabeçalho 1",
    lista_cabecalho2=None,
    alinhamentos=None,
    cores_preenchimento=None,
    colunas_merge=None,
    fonte="Calibri",
    tamanho_fonte=10
):
    # Garantir que os cabeçalhos foram fornecidos
    if lista_cabecalho2 is None:
        raise ValueError("A lista de nomes para o cabeçalho 2 deve ser fornecida.")
    if cores_preenchimento is None:
        cores_preenchimento = ["FFFFFF"] * len(lista_cabecalho2)  # Branco padrão
    if alinhamentos is None:
        alinhamentos = ["center"] * len(lista_cabecalho2)  # Alinhamento central padrão
    if colunas_merge is None:
        colunas_merge = [1] * len(lista_cabecalho2)  # Cada cabeçalho ocupa uma célula por padrão

    # Validar tamanhos das listas
    if not (len(lista_cabecalho2) == len(cores_preenchimento) == len(alinhamentos) == len(colunas_merge)):
        raise ValueError(
            "As listas de alinhamentos, cores de preenchimento e colunas_merge devem ter o mesmo comprimento do cabeçalho 2."
        )

    # Criar um novo workbook e planilha
    wb = Workbook()
    ws = wb.active
    ws.title = "Modelo"

    # Configurar Cabeçalho 1 (linha 1)
    ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=sum(colunas_merge))
    cell_cabecalho1 = ws.cell(row=1, column=1, value=texto_cabecalho1)
    cell_cabecalho1.fill = PatternFill(start_color=cor_cabecalho1, end_color=cor_cabecalho1, fill_type="solid")
    cell_cabecalho1.font = Font(bold=True, name=fonte, size=tamanho_fonte, color="FFFFFF")  # Texto branco obrigatório
    cell_cabecalho1.alignment = Alignment(horizontal="center", vertical="center")

    # Linha 2 (linha oculta)
    ws.row_dimensions[2].hidden = True  # Ocultar a linha 2

    # Configurar Cabeçalho 2 (linha 3)
    col_idx = 1
    for nome, cor_preenchimento, alinhamento, merge in zip(lista_cabecalho2, cores_preenchimento, alinhamentos, colunas_merge):
        # Mesclar as colunas, se necessário
        if merge > 1:
            ws.merge_cells(start_row=3, start_column=col_idx, end_row=3, end_column=col_idx + merge - 1)

        cell = ws.cell(row=3, column=col_idx, value=nome)
        cell.font = Font(color="FFFFFF", bold=True, name=fonte, size=tamanho_fonte)  # Texto branco obrigatório
        cell.fill = PatternFill(start_color=cor_preenchimento, end_color=cor_preenchimento, fill_type="solid")
        cell.alignment = Alignment(horizontal=alinhamento, vertical="center")

        # Ajustar largura das colunas mescladas
        for i in range(merge):
            ws.column_dimensions[get_column_letter(col_idx + i)].width = max(len(nome) + 2, 10)

        col_idx += merge  # Avançar para a próxima célula disponível

    # Salvar o arquivo
    return wb





### preencher_dados_no_modelo

In [65]:
def preencher_dados_no_modelo(wb, ws_name, df, cores_texto=None, negrito=None, bordas=None, alinhamentos=None, colunas_para_mesclar=None, tipos_colunas=None):
    
    # Obter a planilha ativa pelo nome
    if ws_name not in wb.sheetnames:
        raise ValueError(f"A planilha '{ws_name}' não existe no workbook.")
    ws = wb[ws_name]
    
    # Mapear os cabeçalhos 2 com base na linha 3 (após a linha oculta)
    cabecalho_2 = [cell.value for cell in ws[3]]  # Os valores da linha 3 são os cabeçalhos 2
    col_map = {nome: idx + 1 for idx, nome in enumerate(cabecalho_2)}  # Mapear nome para índice de coluna

    # Obter as informações de mesclagem do cabeçalho
    merge_map = {}
    for merged_range in ws.merged_cells.ranges:
        if merged_range.min_row == 3:  # Somente os merges do cabeçalho 2
            merge_map[merged_range.min_col] = merged_range.max_col

    # Validar se todos os nomes do DataFrame estão no cabeçalho 2
    for coluna in df.columns:
        if coluna not in col_map:
            raise ValueError(f"A coluna '{coluna}' no DataFrame não está no cabeçalho 2 do template.")
    
    # Preencher os dados com formatação
    for row_idx, row_data in enumerate(df.itertuples(index=False), start=4):  # Dados começam na linha 4
        
        linha_contem_total = False  # Flag para verificar se a linha contém "Total"
        for col_idx, col_name in enumerate(df.columns):
            cell_idx = col_map[col_name]
            value = row_data[col_idx]
            cell = ws.cell(row=row_idx, column=cell_idx)

            # Verificar se a célula contém "Total" (independente de maiúsculas/minúsculas)
            if isinstance(value, str) and "total" == value.lower():
                linha_contem_total = True
                
            # Determinar o tipo e formatar o valor
            if tipos_colunas and len(tipos_colunas) > col_idx:
                tipo = tipos_colunas[col_idx].lower()
                if tipo == "número" and isinstance(value, (int, float)):
                    cell.value = value
                    cell.number_format = "#,##0.00"
                elif tipo == "moeda" and isinstance(value, (int, float)):
                    cell.value = value
                    cell.number_format = "R$ #,##0.00"
                else:  # Default: texto
                    cell.value = str(value)
            else:
                cell.value = value
            
            # Replicar mesclagem do cabeçalho, se aplicável
            if cell_idx in merge_map:
                start_col = cell_idx
                end_col = merge_map[cell_idx]
                ws.merge_cells(
                    start_row=row_idx, start_column=start_col, end_row=row_idx, end_column=end_col
                )
                # Aplicar 'Wrap Text' à célula mesclada
                cell = ws.cell(row=row_idx, column=start_col)
                cell.alignment = Alignment(horizontal="center", vertical="center",wrap_text=True)

            # Aplicar formatações, se fornecidas
            font_color = cores_texto[col_idx] if cores_texto and len(cores_texto) > col_idx else "000000"
            is_bold = negrito[col_idx] if negrito and len(negrito) > col_idx else False
            
            
            # Alterar a cor do texto para vermelho se o valor começar com "-"
            if (isinstance(value, str) and value.startswith("-")) or ((isinstance(value, float) or isinstance(value, int)) and value<0):
                font_color="FF0000"
                
            cell_font = Font(color=font_color, bold=is_bold)
            cell.font = cell_font

            # Aplicar bordas, se fornecidas
            if bordas and len(bordas) > col_idx and bordas[col_idx]:
                dotted_border = Border(
                    bottom=Side(style="dotted"),
                    top=Side(style="dotted"),
                    left=Side(style="dotted"),
                    right=Side(style="dotted")
                )
                cell.border = dotted_border

            # Aplicar alinhamentos, se fornecidos
            alignment_style = Alignment(horizontal="center")
            if alinhamentos and len(alinhamentos) > col_idx:
                alinhamento = alinhamentos[col_idx].lower()
                if alinhamento in ["left", "right", "center"]:
                    alignment_style = Alignment(horizontal=alinhamento)
            
            cell.alignment = alignment_style
        # Aplicar o estilo negrito em toda a linha, se "Total" estiver presente
        if linha_contem_total:
            for col_idx in range(1, len(df.columns) + 1):
                cell = ws.cell(row=row_idx, column=col_idx)
                cell.font = Font(bold=True,color=cell.font.color)
    
    # Mesclar células em colunas específicas para valores iguais
    if colunas_para_mesclar:
        for coluna in colunas_para_mesclar:
            col_idx = col_map[coluna]
            start_row = 4  # Dados começam na linha 4
            value_to_merge = None
            merge_start = None
            
            for row in range(start_row, start_row + len(df)):
                cell = ws.cell(row=row, column=col_idx)
                if cell.value == value_to_merge:
                    # Continuar mesclagem
                    continue
                else:
                    # Finalizar mesclagem anterior
                    if merge_start and merge_start != row - 1:
                        ws.merge_cells(
                            start_row=merge_start, start_column=col_idx, 
                            end_row=row - 1, end_column=col_idx
                        )
                        merged_cell = ws.cell(row=merge_start, column=col_idx)
                        merged_cell.alignment = Alignment(horizontal="center", vertical="center",wrap_text=True)
                        # Aplicar 'Wrap Text' à célula mesclada
                        # Garantir que o tipo e formato da célula mesclada seja o mesmo
                        if tipos_colunas and len(tipos_colunas) > col_idx:
                            tipo = tipos_colunas[col_idx].lower()
                            if tipo == "número":
                                merged_cell.number_format = "#,##0.00"
                            elif tipo == "moeda":
                                merged_cell.number_format = "R$ #,##0.00"
                    
                    # Iniciar nova mesclagem
                    value_to_merge = cell.value
                    merge_start = row
            
            # Finalizar a última mesclagem
            if merge_start and merge_start != (start_row + len(df) - 1):
                ws.merge_cells(
                    start_row=merge_start, start_column=col_idx, 
                    end_row=start_row + len(df) - 1, end_column=col_idx
                )
                merged_cell = ws.cell(row=merge_start, column=col_idx)
                merged_cell.alignment = Alignment(horizontal="center", vertical="center",wrap_text=True)
                # Garantir que o tipo e formato da célula mesclada seja o mesmo
                if tipos_colunas and len(tipos_colunas) > col_idx:
                    tipo = tipos_colunas[col_idx].lower()
                    if tipo == "número":
                        merged_cell.number_format = "#,##0.00"
                    elif tipo == "moeda":
                        merged_cell.number_format = "R$ #,##0.00"

### unir_workbooks_em_xlsx

In [66]:
def unir_workbooks_em_xlsx(lista_wb, path_arquivo_base, destino, nome_novo, celula_inicial="D9"):
    
    wb_base = load_workbook(path_arquivo_base)
    ws_base = wb_base.active

    # Determinar linha e coluna inicial a partir da célula inicial
    col_inicial = celula_inicial[0].upper()
    row_inicial = int(celula_inicial[1:])
    col_inicial_idx = ord(col_inicial) - ord('A') + 1

    linha_atual = row_inicial  # Começa pela linha inicial especificada
    
    # Unir workbooks
    for wb in lista_wb:
        ws = wb.active  # Assume que o conteúdo está na aba ativa do workbook

        # Determinar o maior índice de coluna usado no workbook atual
        colunas = ws.max_column

        # Copiar visibilidade de colunas
        for col_idx in range(1, colunas + 1):
            col_letter = get_column_letter(col_idx)
            ws_base.column_dimensions[col_letter].hidden = ws.column_dimensions[col_letter].hidden

        # Copiar visibilidade de linhas
        for row_idx in range(1, ws.max_row + 1):
            if ws.row_dimensions[row_idx].hidden:
                ws_base.row_dimensions[linha_atual + row_idx - 1].hidden = True

        # Copiar conteúdo e estilos de células
        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, max_col=colunas):
            for col_idx, cell in enumerate(row, start=col_inicial_idx):
                nova_celula = ws_base.cell(row=linha_atual, column=col_idx)
                nova_celula.value = cell.value

                # Copiar estilo, recriando os objetos de estilo
                if cell.has_style:
                    nova_celula.font = Font(
                        name=cell.font.name,
                        size=cell.font.size,
                        bold=cell.font.bold,
                        italic=cell.font.italic,
                        underline=cell.font.underline,
                        strike=cell.font.strike,
                        color=cell.font.color
                    )
                    nova_celula.fill = PatternFill(
                        fill_type=cell.fill.fill_type,
                        start_color=cell.fill.start_color,
                        end_color=cell.fill.end_color
                    )
                    nova_celula.border = Border(
                        left=cell.border.left,
                        right=cell.border.right,
                        top=cell.border.top,
                        bottom=cell.border.bottom
                    )
                    nova_celula.alignment = Alignment(
                        horizontal=cell.alignment.horizontal,
                        vertical=cell.alignment.vertical,
                        wrap_text=cell.alignment.wrap_text,
                        shrink_to_fit=cell.alignment.shrink_to_fit,
                        indent=cell.alignment.indent
                    )
                    nova_celula.number_format = cell.number_format

            linha_atual += 1
        for col_idx in range(1, colunas + 1):
            col_letter = get_column_letter(col_idx)
            ws_base.column_dimensions[col_letter].hidden = ws.column_dimensions[col_letter].hidden

        # Copiar visibilidade de linhas
        for row_idx in range(1, ws.max_row + 1):
            if ws.row_dimensions[row_idx].hidden:
                ws_base.row_dimensions[linha_atual + row_idx - 1].hidden = True
        # Copiar células mescladas
        for merged_cell_range in ws.merged_cells.ranges:
            # Traduzir o intervalo para a nova posição
            min_col, min_row, max_col, max_row = range_boundaries(str(merged_cell_range))
            min_col += (col_inicial_idx - 1)  # Ajustar para a coluna inicial
            max_col += (col_inicial_idx - 1)
            min_row += (linha_atual - ws.max_row - 1)  # Ajustar para a linha atual
            max_row += (linha_atual - ws.max_row - 1)
            new_range = f"{get_column_letter(min_col)}{min_row}:{get_column_letter(max_col)}{max_row}"
            ws_base.merge_cells(new_range)
            # cell = ws.cell(row=min_row, column=min_col)
            # cell.alignment = Alignment(horizontal="center", vertical="center",wrap_text=True)

        # Adicionar uma linha em branco como separador
        linha_atual += 1
        
    # Adicionar borda três linhas abaixo da última linha
    linha_borda = linha_atual + 2
    borda_inferior = Border(bottom=Side(style="thin"))
    for col in range(4, 10):  # Colunas C a J (índices 3 a 10)
        celula = ws_base.cell(row=linha_borda, column=col)
        celula.border = borda_inferior
        
    borda_esquerda = Border(left=Side(style="thin"),bottom=Side(style="thin"))
    celula = ws_base.cell(row=linha_borda, column=3)
    celula.border = borda_esquerda
    
    borda_direita = Border(right=Side(style="thin"),bottom=Side(style="thin"))
    celula = ws_base.cell(row=linha_borda, column=10)
    celula.border = borda_direita
    
    thin = Side(style="thin")

    # Loop de primeira até a linha_borda (incluindo ela)
    for row in range(row_inicial, linha_borda):
        # Coluna C = 3 -> borda ESQUERDA
        cell_c = ws_base.cell(row=row, column=3)
        cell_c.border = Border(left=thin)
        
        # Coluna J = 10 -> borda DIREITA
        cell_j = ws_base.cell(row=row, column=10)
        cell_j.border = Border(right=thin)
    
    # Remover formatação de todas as células abaixo da linha com a borda
    for row in ws_base.iter_rows(min_row=linha_borda + 1, max_row=ws_base.max_row, max_col=ws_base.max_column):
        for cell in row:
            cell.value = None
            cell.font = Font()  # Font padrão (reset)
            cell.fill = PatternFill()  # Sem preenchimento
            cell.border = Border()  # Sem borda
            cell.alignment = Alignment()  # Alinhamento padrão
    
    # Salvar o arquivo final
    caminho_final = Path.joinpath(destino,nome_novo)
    wb_base.save(caminho_final)
    #print(f"\nArquivo salvo em: {caminho_final}")
    return caminho_final




### formatar_cpf_cnpj

In [67]:
def formatar_cpf_cnpj(cpf_cnpj):
    if pd.isnull(cpf_cnpj):
        return cpf_cnpj
    cpf_cnpj = str(cpf_cnpj).strip()
    cpf_cnpj = ''.join(filter(str.isdigit, cpf_cnpj))  # Mantém apenas números

    if len(cpf_cnpj) == 14:  # CNPJ
        return f"{cpf_cnpj[:2]}.{cpf_cnpj[2:5]}.{cpf_cnpj[5:8]}/{cpf_cnpj[8:12]}-{cpf_cnpj[12:]}"
    elif len(cpf_cnpj) == 11:  # CPF
        return f"{cpf_cnpj[:3]}.{cpf_cnpj[3:6]}.{cpf_cnpj[6:9]}-{cpf_cnpj[9:]}"
    elif len(cpf_cnpj) < 11:  # CPF incompleto, preencher com zeros
        cpf_cnpj = cpf_cnpj.zfill(11)
        return f"{cpf_cnpj[:3]}.{cpf_cnpj[3:6]}.{cpf_cnpj[6:9]}-{cpf_cnpj[9:]}"
    elif len(cpf_cnpj) < 14:  # CNPJ incompleto, preencher com zeros
        cpf_cnpj = cpf_cnpj.zfill(14)
        return f"{cpf_cnpj[:2]}.{cpf_cnpj[2:5]}.{cpf_cnpj[5:8]}/{cpf_cnpj[8:12]}-{cpf_cnpj[12:]}"
    
    return cpf_cnpj  # Retorna o valor original se não for possível formatar

### format_date_columns

In [68]:
def format_date_columns(df, date_columns):
    for col in date_columns:
        if col in df.columns:
            # Converte em bloco, assumindo formato DD/MM/YYYY, força coerção de erros
            df[col] = (
                pd.to_datetime(df[col], dayfirst=True, format='%d/%m/%Y', errors='coerce')
                  .dt.strftime('%d/%m/%Y')
                  .fillna('')
            )
        else:
            print(f"Warning: Column '{col}' does not exist in the DataFrame.")
    return df


### formatar_valores

In [69]:
import locale
def formatar_valores(valor, moeda=False):
    
    if pd.isnull(valor):
        return valor
    valor = str(valor)
    try:
        # Configura o locale para o Brasil
        locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
        
        
        # Remove pontos e converte vírgulas em pontos para valores com separador brasileiro
        if "," in valor and "." in valor:
            valor = valor.replace(".", "").replace(",", ".")
        elif "," in valor:
            valor = valor.replace(",", ".")
        
        # Converte para float
        try:
            numero = float(valor)
        except ValueError:
            raise ValueError(f"Não foi possível converter o valor: {valor}")
        
        # Formata o número no padrão monetário brasileiro
        valor_formatado = locale.currency(abs(numero), grouping=True, symbol=True)
        
        # Adiciona o sinal negativo para números negativos
        if numero < 0:
            valor_formatado = valor_formatado.replace("R$", "- R$")
        
        return valor_formatado if moeda else numero
    except Exception as e:
        # print(f"Erro ao formatar o valor '{valor}': {e}")
        return valor

### to_pascal_case

In [70]:
def to_pascal_case(s):
    if pd.isnull(s):
        return s
    return ' '.join(word.capitalize() for word in s.split(' '))

### xlsx_to_pdf_one_page

In [71]:
def xlsx_to_pdf_one_page(input_file, output_path):


    os.makedirs(output_path,exist_ok=True)
    filename = os.path.splitext(os.path.basename(input_file))[0] + ".pdf"
    
    output_file = Path(output_path, filename)
    
    if os.path.isfile(output_file):
        return
    
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.Visible = False

    try:
        wb = excel.Workbooks.Open(input_file)
    except Exception as e:
        print(f"Erro ao abrir o arquivo {input_file}: {e}")
        return

    ws = wb.Worksheets(1)  # planilha COM
    ws.Columns.AutoFit()

    # área de impressão de C3 até última coluna/linha
    last_column = 12
    last_row = 1
    for row in range(1, ws.UsedRange.Rows.Count + 1):
        borders = ws.Cells(row, 4).Borders
        for border_id in range(5, 13):
            if borders(border_id).Color != 0.0 or borders(border_id).LineStyle != -4142:
                last_row = row
                break    

    for col in range(1, ws.UsedRange.Columns.Count + 1):
        borders = ws.Cells(5, col).Borders
        for border_id in range(5, 13):
            if borders(border_id).Color != 0.0 or borders(border_id).LineStyle != -4142:
                # last_column = col
                break

    def column_index_to_letter(index):
        letter = ''
        while index > 0:
            index, rem = divmod(index - 1, 26)
            letter = chr(65 + rem) + letter
        return letter

    last_col_letter = column_index_to_letter(last_column)
    ws.PageSetup.PrintArea = ws.Range(f"C3:{last_col_letter}{last_row}").Address

    # configura impressão em 1 página
    ws.PageSetup.Zoom = False
    ws.PageSetup.FitToPagesWide = 1
    ws.PageSetup.FitToPagesTall = 1


    # **Auto-ajusta a largura de todas as colunas usadas**

    # exporta pra PDF
    wb.ExportAsFixedFormat(0, str(output_file))

    wb.Close(False)
    excel.Quit()
    #print(f"\nArquivo PDF gerado: {output_file}")

### Processamento dos Quadros

#### processar_quadro_zero

In [72]:
def processar_quadro_zero(df_quadro_zero, status_cols, contrato, comparativo):
    df_quadro = df_quadro_zero.T
    df_quadro.reset_index(inplace=True)
    df_quadro.columns = ["Ref.","Base Analítica"]
    df_quadro["Contrato/Tela Sistêmica"] = ""
    
    conferencias = comparativo[comparativo["Contrato"]==contrato].drop_duplicates()[status_cols]
    conferencias["Contrato"] = "Conferido"
    conferencias = conferencias.T.reset_index()
    conferencias.columns = ["Ref.","Base Analítica x Contrato/Tela Sistêmica"]
        
    df_quadro["Base Analítica x Contrato/Tela Sistêmica"] = conferencias["Base Analítica x Contrato/Tela Sistêmica"]
    df_quadro["Anexo"] = f"Anexo N_A{contrato}"
    
    df_quadro["Contrato/Tela Sistêmica"] = df_quadro.apply(lambda x: x["Base Analítica"] if x["Base Analítica x Contrato/Tela Sistêmica"]== "Conferido" else "",axis=1)
    df_quadro["Base Analítica x Contrato/Tela Sistêmica"] = df_quadro["Base Analítica x Contrato/Tela Sistêmica"].apply(lambda x: "Divergente" if x!= "Conferido" else x)
    
    df_quadro["Base Analítica"] = df_quadro.apply(lambda x: formatar_valores(x["Base Analítica"], moeda=True) if "Valor do Bem" in x["Ref."] else x["Base Analítica"],axis=1)
    df_quadro["Contrato/Tela Sistêmica"] = df_quadro.apply(lambda x: formatar_valores(x["Contrato/Tela Sistêmica"], moeda=True) if "Valor do Bem" in x["Ref."] else x["Contrato/Tela Sistêmica"],axis=1)

    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1="RESUMO DAS VALIDAÇÕES DAS OPERAÇÕES DE LEASING - ITAÚ UNIBANCO S.A.",
        lista_cabecalho2=["Ref.","Base Analítica","Contrato/Tela Sistêmica","Base Analítica x Contrato/Tela Sistêmica","Anexo"],
        alinhamentos=["center", "center", "center", "center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 2, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )

    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro, 
                                cores_texto=[None, None, None, None, "FF0000"], 
                                negrito=[False, False, False, False, True], 
                                bordas=[True, True, True, True, True],
                                alinhamentos=["left", "center", "center", "right" , "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "texto", "texto", "texto","texto"])
    
    return workbook

#### processar_quadro_um

In [73]:
def processar_quadro_um(df_contratuais_vs_registro_contabil, contrato):
    df_quadro = df_contratuais_vs_registro_contabil[df_contratuais_vs_registro_contabil["Contrato"]==contrato]
    
    df_quadro["Valor do ativo contabilizado"] = df_quadro["Valor do ativo contabilizado"].apply( lambda x: np.nan if x == "0.00" else x)
    df_quadro["Valores residuais a realizar contabilizado"] = df_quadro["Valores residuais a realizar contabilizado"].apply( lambda x: np.nan if x == "0.00" else x)
    df_quadro["Total Contraprestação de Arrendamento - CPA"] = df_quadro["Total Contraprestação de Arrendamento - CPA"].apply( lambda x: np.nan if x == "0.00" else x)
    df_quadro["Valor do ativo contabilizado"] = df_quadro["Valor do ativo contabilizado"].apply( lambda x: np.nan if x == "0.0000" else x)
    df_quadro["Valores residuais a realizar contabilizado"] = df_quadro["Valores residuais a realizar contabilizado"].apply( lambda x: np.nan if x == "0.0000" else x)
    df_quadro["Total Contraprestação de Arrendamento - CPA"] = df_quadro["Total Contraprestação de Arrendamento - CPA"].apply( lambda x: np.nan if x == "0.0000" else x)
    
    df_quadro_pivot = df_quadro.melt(
    id_vars=["Contrato", "COSIF"],  # Colunas que permanecerão como estão
    value_vars=[
        "Valor do ativo contabilizado",
        "Valores residuais a realizar contabilizado",
        "Total Contraprestação de Arrendamento - CPA",
    ],  # Colunas a serem transformadas
    var_name="Descrição",  # Nome da nova coluna com os nomes das descrições
    value_name="Valor",  # Nome da nova coluna com os valores
)
    df_quadro_pivot.dropna(subset=["Valor"], inplace=True)
    # Filtrar apenas as linhas com valores diferentes de zero, se necessário
    df_quadro_pivot = df_quadro_pivot[df_quadro_pivot["Valor"] != 0]
    df_quadro_pivot = df_quadro_pivot[["Descrição","COSIF","Valor"]]
    df_quadro_pivot.columns = ["Descrição","COSIF","Valor Contabilizado"]
    df_quadro_pivot["Anexo"] = f"B{contrato}"# f"{name}_DADOS CONTRATUAIS X REGISTRO CONTÁBIL.xlsx"
    
    df_quadro_pivot["Valor Contabilizado"] = df_quadro_pivot["Valor Contabilizado"].apply(formatar_valores)
    
    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"DADOS CONTRATUAIS X REGISTRO CONTÁBIL",
        lista_cabecalho2=["Descrição","COSIF","Valor Contabilizado","Anexo"],
        alinhamentos=["center", "center", "center", "center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060"],
        colunas_merge=[1, 1, 3, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )
    
    preencher_dados_no_modelo(workbook,
                                "Modelo", 
                                df_quadro_pivot, 
                                cores_texto=[None, None, None, "FF0000"], 
                                negrito=[False, False, False, True], 
                                bordas=[True, True, True, True],
                                alinhamentos=["left", "center", "right", "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "texto", "moeda", "texto"])
    
    return workbook

#### processar_quadro_dois

In [74]:
def processar_quadro_dois(df_demonstracao_valor_anual, contrato):

    df_quadro = df_demonstracao_valor_anual[df_demonstracao_valor_anual["Contrato"]==contrato]
    
    df_quadro["Anexo"] = f"B{contrato}"
    
    df_quadro.columns = ["Contrato","Ano","COSIF","Valor Líquido Contabilizado","Descrição","Anexo"]
    df_quadro.fillna("",inplace=True)
    df_quadro["Valor Líquido Contabilizado"]=df_quadro["Valor Líquido Contabilizado"].apply(formatar_valores)
    
    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"DEMONSTRAÇÃO DO VALOR ANUAL DA SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO CONTABILIZADOS NO PERÍODO  - CONTRATO {contrato}",
        lista_cabecalho2=["Ano","COSIF","Descrição","Valor Líquido Contabilizado","Anexo"],
        alinhamentos=["center", "center", "center", "center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 2, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )
    df_quadro = df_quadro[["Ano","COSIF","Descrição","Valor Líquido Contabilizado","Anexo"]]
    # Chamar a função para preencher os dados com formatação
    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","COSIF","Descrição","Valor Líquido Contabilizado","Anexo"]], 
                                cores_texto=["FF0000", None, None, None, "FF0000"], 
                                negrito=[True, False, False, False, True], 
                                bordas=[True, True, True, True, True],
                                alinhamentos=["center", "center", "center", "right" , "center"],
                                colunas_para_mesclar=["COSIF","Descrição","Anexo"],
                                tipos_colunas=["texto", "texto", "texto", "moeda", "texto"])
    
    return workbook

#### processar_quadro_tres

In [75]:
def processar_quadro_tres(df_efeito_resultado_contabil, contrato):
    df_quadro = df_efeito_resultado_contabil[df_efeito_resultado_contabil["Contrato"]==contrato]
    # df_efeito_resultado_contabil_filtrado.to_excel(f"projects_data/Itau/OneDrive_2024-10-21/tela_preta/itens_96/{name}/{name}_RESULTADO CONTÁBIL LÍQUIDO DO CONTRATO COM EFEITO DA SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO CONTABILIZADOS NO PERÍODO.xlsx",index=False)
    df_quadro["Anexo"] = f"C{contrato}"#f"{name}_RESULTADO CONTÁBIL LÍQUIDO DO CONTRATO COM EFEITO DA SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO CONTABILIZADOS NO PERÍODO.xlsx"
    df_quadro.columns = ["Contrato","Receita de Contraprestação - Inclui Superveniência (A)","Despesa de Depreciação - Inclui Insuficiência (B)","Descontos Concedidos (C)","LAIR (A) + (B) + (C) = (D)","Ano","Anexo"]
    df_quadro["Receita de Contraprestação - Inclui Superveniência (A)"]=df_quadro["Receita de Contraprestação - Inclui Superveniência (A)"].apply(formatar_valores)
    df_quadro["Despesa de Depreciação - Inclui Insuficiência (B)"]=df_quadro["Despesa de Depreciação - Inclui Insuficiência (B)"].apply(formatar_valores)
    df_quadro["Descontos Concedidos (C)"]=df_quadro["Descontos Concedidos (C)"].apply(formatar_valores)
    df_quadro["LAIR (A) + (B) + (C) = (D)"]=df_quadro["LAIR (A) + (B) + (C) = (D)"].apply(formatar_valores)
    
    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"EFEITOS NO RESULTADO CONTÁBIL RELATIVOS AO CONTRATO DE ARRENDAMENTO NÚMERO {contrato}",
        lista_cabecalho2=["Ano","Receita de Contraprestação - Inclui Superveniência (A)","Despesa de Depreciação - Inclui Insuficiência (B)","Descontos Concedidos (C)","LAIR (A) + (B) + (C) = (D)","Anexo"],
        alinhamentos=["center", "center", "center", "center","center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 1, 1, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )

    # Chamar a função para preencher os dados com formatação
    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","Receita de Contraprestação - Inclui Superveniência (A)","Despesa de Depreciação - Inclui Insuficiência (B)","Descontos Concedidos (C)","LAIR (A) + (B) + (C) = (D)","Anexo"]], 
                                cores_texto=["FF0000", None, None, None,None, "FF0000"], 
                                negrito=[True, False, False, False, False, True], 
                                bordas=[True, True, True, True, True,True],
                                alinhamentos=["center", "right", "right", "right" , "right", "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "moeda", "moeda", "moeda", "moeda", "texto"])
    
    return workbook

#### processar_quadro_quatro

In [76]:
def processar_quadro_quatro(df_base_calculo_irpj, contrato):
    df_quadro = df_base_calculo_irpj[df_base_calculo_irpj["Contrato"]==contrato]
    # df_base_calculo_irpj_filtrado.to_excel(f"projects_data/Itau/OneDrive_2024-10-21/tela_preta/itens_96/{name}/{name}_BASE DE CÁLCULO DO IRPJ RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx",index=False)
    df_quadro["Anexo"] = f"C{contrato}"#f"{name}_BASE DE CÁLCULO DO IRPJ RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx"
    df_quadro.columns = ["Contrato","Ano","RESULTADO ANTES DO IRPJ","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo do IRPJ","Anexo"]
    df_quadro["RESULTADO ANTES DO IRPJ"]=df_quadro["RESULTADO ANTES DO IRPJ"].apply(formatar_valores)
    df_quadro["ADIÇÕES -  Descontos Concedidos"]=df_quadro["ADIÇÕES -  Descontos Concedidos"].apply(formatar_valores)
    df_quadro["Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação"]=df_quadro["Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação"].apply(formatar_valores)
    df_quadro["Base de Cálculo do IRPJ"]=df_quadro["Base de Cálculo do IRPJ"].apply(formatar_valores)
    
    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"BASE DE CÁLCULO DO IRPJ RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO NÚMERO {contrato}",
        lista_cabecalho2=["Ano","RESULTADO ANTES DO IRPJ","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo do IRPJ","Anexo"],
        alinhamentos=["center", "center", "center", "center","center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 1, 1, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )
    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","RESULTADO ANTES DO IRPJ","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo do IRPJ","Anexo"]], 
                                cores_texto=["FF0000", None, None, None,None, "FF0000"], 
                                negrito=[True, False, False, False, False, True], 
                                bordas=[True, True, True, True, True,True],
                                alinhamentos=["center", "right", "right", "right" , "right", "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "moeda", "moeda", "moeda", "moeda", "texto"])
    
    return workbook

#### processar_quadro_cinco

In [77]:
def processar_quadro_cinco(df_base_calculo_csll, contrato):
    df_quadro = df_base_calculo_csll[df_base_calculo_csll["Contrato"]==contrato]
    # df_base_calculo_csll_filtrado.to_excel(f"projects_data/Itau/OneDrive_2024-10-21/tela_preta/itens_96/{name}/{name}_BASE DE CÁLCULO DA CSLL RELATIVA À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx",index=False)
    df_quadro["Anexo"] = f"C{contrato}"#f"{name}_BASE DE CÁLCULO DA CSLL RELATIVA À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx"
    df_quadro.columns = ["Contrato","Ano","RESULTADO ANTES DA CSLL","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo da CSLL","Anexo"]
    df_quadro["RESULTADO ANTES DA CSLL"]=df_quadro["RESULTADO ANTES DA CSLL"].apply(formatar_valores)
    df_quadro["ADIÇÕES -  Descontos Concedidos"]=df_quadro["ADIÇÕES -  Descontos Concedidos"].apply(formatar_valores)
    df_quadro["Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação"]=df_quadro["Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação"].apply(formatar_valores)
    df_quadro["Base de Cálculo da CSLL"]=df_quadro["Base de Cálculo da CSLL"].apply(formatar_valores)

    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"BASE DE CÁLCULO DA CSLL RELATIVA À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO NÚMERO {contrato}",
        lista_cabecalho2=["Ano","RESULTADO ANTES DA CSLL","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo da CSLL","Anexo"],
        alinhamentos=["center", "center", "center", "center","center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 1, 1, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )
    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","RESULTADO ANTES DA CSLL","ADIÇÕES -  Descontos Concedidos","Adições/(Exclusões) - Superveniência/Insuficiência de Depreciação","Base de Cálculo da CSLL","Anexo"]], 
                                cores_texto=["FF0000", None, None, None,None, "FF0000"], 
                                negrito=[True, False, False, False, False, True], 
                                bordas=[True, True, True, True, True,True],
                                alinhamentos=["center", "right", "right", "right" , "right", "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "moeda", "moeda", "moeda", "moeda", "texto"])
    
    return workbook 

#### processar_quadro_seis

In [78]:
def processar_quadro_seis(df_demonstrativo_efeito_nulo, contrato):
    df_quadro = df_demonstrativo_efeito_nulo[df_demonstrativo_efeito_nulo["Contrato"]==contrato]
    # df_demonstrativo_efeito_nulo_filtrado.to_excel(f"projects_data/Itau/OneDrive_2024-10-21/tela_preta/itens_96/{name}/{name}_DEMONSTRAÇÃO DO EFEITO NULO DOS VALORES DE SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO NAS BASES DO IRPJ E DA CSLL RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx",index=False)
    df_quadro["Anexo"] = f"C{contrato}"#f"{name}_DEMONSTRAÇÃO DO EFEITO NULO DOS VALORES DE SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO NAS BASES DO IRPJ E DA CSLL RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO.xlsx"
    df_quadro.columns = ["Contrato","Ano","BASE DO IRPJ","BASE DA CSLL","DIFERENÇA","SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO","Anexo"]
    df_quadro["BASE DO IRPJ"]=df_quadro["BASE DO IRPJ"].apply(formatar_valores)
    df_quadro["BASE DA CSLL"]=df_quadro["BASE DA CSLL"].apply(formatar_valores)
    df_quadro["DIFERENÇA"]=df_quadro["DIFERENÇA"].apply(formatar_valores)
    df_quadro["SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO"]=df_quadro["SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO"].apply(formatar_valores)
    workbook = criar_modelo_excel(
        nome_arquivo="Modelo_Com_Linha_Oculta.xlsx",
        cor_cabecalho1="002060",
        texto_cabecalho1=f"DEMONSTRAÇÃO DO EFEITO NULO DOS VALORES DE SUPERVENIÊNCIA E INSUFICIÊNCIA DE DEPRECIAÇÃO NAS BASES DO IRPJ E DA CSLL RELATIVO À OPERAÇÃO DO CONTRATO DE ARRENDAMENTO NÚMERO {contrato}",
        lista_cabecalho2=["Ano","BASE DO IRPJ","BASE DA CSLL","DIFERENÇA","SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO","Anexo"],
        alinhamentos=["center", "center", "center", "center","center","center"],  # Alinhamentos específicos por coluna
        cores_preenchimento=["002060", "002060","002060","002060","002060","002060"],
        colunas_merge=[1, 1, 1, 1, 1, 1],
        fonte="Calibri",
        tamanho_fonte=10
    )
    preencher_dados_no_modelo(workbook, 
                                "Modelo", 
                                df_quadro[["Ano","BASE DO IRPJ","BASE DA CSLL","DIFERENÇA","SUPERVENIÊNCIA/INSUFICIÊNCIA DE DEPRECIAÇÃO","Anexo"]], 
                                cores_texto=["FF0000", None, None, None,None, "FF0000"], 
                                negrito=[True, False, False, False, False, True], 
                                bordas=[True, True, True, True, True,True],
                                alinhamentos=["center", "right", "right", "right" , "right", "center"],
                                colunas_para_mesclar=["Anexo"],
                                tipos_colunas=["texto", "moeda", "moeda", "moeda", "moeda", "texto"])
    
    return workbook

### criar_folha_rosto

In [79]:
def criar_folha_rosto( contrato
                     , df_quadro_zero
                     , comparativo
                     , caminho_destino
                     , df_contratuais_vs_registro_contabil
                     , df_demonstracao_valor_anual
                     , df_efeito_resultado_contabil
                     , df_base_calculo_irpj
                     , df_base_calculo_csll
                     , df_demonstrativo_efeito_nulo
                     , caminho_base
                     , replace=False ):
    
    nome_final = f"RESUMO_{contrato}.xlsx"

    if not replace and os.path.isfile(Path.joinpath(caminho_destino,nome_final)):
        return
    os.makedirs(caminho_destino,exist_ok=True)
    
    wb_resumo = processar_quadro_zero(df_quadro_zero, status_cols, contrato, comparativo)
    wb_contratuais_vs_registro_contabil = processar_quadro_um(df_contratuais_vs_registro_contabil, contrato)
    wb_demonstracao_valor_anual = processar_quadro_dois(df_demonstracao_valor_anual, contrato)
    wb_efeito_resultado_contabil = processar_quadro_tres(df_efeito_resultado_contabil, contrato)
    wb_base_calculo_irpj = processar_quadro_quatro(df_base_calculo_irpj, contrato)
    wb_base_calculo_csll = processar_quadro_cinco(df_base_calculo_csll, contrato)
    wb_demonstrativo_efeito_nulo = processar_quadro_seis(df_demonstrativo_efeito_nulo, contrato)
    
    unir_workbooks_em_xlsx(
        lista_wb=[wb_resumo, 
                    wb_contratuais_vs_registro_contabil,
                    wb_demonstracao_valor_anual,
                    wb_efeito_resultado_contabil,
                    wb_base_calculo_irpj,
                    wb_base_calculo_csll,
                    wb_demonstrativo_efeito_nulo
                    ],
        path_arquivo_base=caminho_base,
        destino=caminho_destino,
        nome_novo=nome_final,
        celula_inicial="D9"
    )


### process_anexo_pdfs

In [80]:
def process_anexo_pdfs(caminho_base, limite_mb=5, contador_inicial=0):
    # Caminho do arquivo .xlsx
    # file_path = "filtro2.xlsx"
    # data = pd.read_excel(file_path)
    # data["Contrato"] = data["Contrato"].astype(str)
    # contratos_invalidos = data["Contrato"].values.tolist()
    contratos_invalidos = []  # Zerado conforme o código original
    limite_bytes = limite_mb * 1024 * 1024
    lote = os.path.basename(os.path.dirname(caminho_base))
    anexo = os.path.basename(caminho_base)

    arquivos_info = []
    for f in os.listdir(caminho_base):
        if f.endswith(".pdf"):
            caminho_pdf = os.path.join(caminho_base, f)
            tamanho = os.path.getsize(caminho_pdf)
            # Verifica se o número do contrato (apenas dígitos do nome) não está na lista de contratos inválidos
            if re.sub(r'\D', '', f) not in contratos_invalidos:
                arquivos_info.append((re.sub(r'\D', '', f), caminho_pdf, tamanho))

    # Cria a pasta de saída para os lotes
    pasta_final = f"{caminho_base}_{lote}_partes"
    os.makedirs(pasta_final, exist_ok=True)

    # Função para dividir PDFs que excedem o tamanho máximo permitido
    def dividir_pdf(origem, tamanho_max, label):
        partes = []
        with open(origem, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            writer = None
            parte_indice = 1
            tamanho_acumulado = 0
            for page in reader.pages:
                temp_writer = PyPDF2.PdfWriter()
                temp_writer.add_page(page)
                # Salva temporariamente para verificar o tamanho da página
                with open("temp_part.pdf", "wb") as temp_out:
                    temp_writer.write(temp_out)
                parte_tamanho = os.path.getsize("temp_part.pdf")
                if not writer or (tamanho_acumulado + parte_tamanho > tamanho_max):
                    if writer:
                        nome_parte = os.path.join(pasta_final, f"{label}_parte_{parte_indice}.pdf")
                        with open(nome_parte, "wb") as out:
                            writer.write(out)
                        partes.append(nome_parte)
                        parte_indice += 1
                    writer = PyPDF2.PdfWriter()
                    tamanho_acumulado = 0
                writer.add_page(page)
                tamanho_acumulado += parte_tamanho
            if writer:
                nome_parte = os.path.join(pasta_final, f"{label}_parte_{parte_indice}.pdf")
                with open(nome_parte, "wb") as out:
                    writer.write(out)
                partes.append(nome_parte)
        os.remove("temp_part.pdf")
        return partes

    # Agrupa os PDFs (ou partes deles) em grupos cujo tamanho total não exceda 'limite_bytes'
    grupos = []
    grupo_atual = []
    tamanho_atual = 0
    for pasta, caminho_pdf, tamanho in arquivos_info:
        if tamanho > limite_bytes:
            nome_base = os.path.splitext(os.path.basename(caminho_pdf))[0]
            partes = dividir_pdf(caminho_pdf, limite_bytes, nome_base)
            for parte in partes:
                p_size = os.path.getsize(parte)
                parte_nome = os.path.basename(parte)
                if p_size + tamanho_atual <= limite_bytes:
                    grupo_atual.append((pasta, parte_nome))
                    tamanho_atual += p_size
                else:
                    if grupo_atual:
                        grupos.append(grupo_atual)
                    grupo_atual = [(pasta, parte_nome)]
                    tamanho_atual = p_size
        else:
            if tamanho + tamanho_atual <= limite_bytes:
                grupo_atual.append((pasta, caminho_pdf))
                tamanho_atual += tamanho
            else:
                if grupo_atual:
                    grupos.append(grupo_atual)
                grupo_atual = [(pasta, caminho_pdf)]
                tamanho_atual = tamanho
    if grupo_atual:
        grupos.append(grupo_atual)

    dados_excel = []
    final_pdfs = []  # Lista para armazenar os caminhos dos PDFs finais (lotes)
    for i, grupo in enumerate(tqdm(grupos, desc="Processando grupos"), start=1):
        # Registra informações para o Excel (sem criar o TXT)
        for (p, arquivo) in grupo:
            parte = f" - Parte {arquivo.split('_parte_')[-1].split('.pdf')[0]}" if "_parte_" in arquivo else ""
            dados_excel.append({"Contrato Número": p, "Lote": f"{anexo}_{lote}_parte_{i}.pdf", "Parte do contrato": parte})

        # Cria o PDF final para o grupo
        escritor_final = PyPDF2.PdfWriter()
        for (p, arquivo) in grupo:
            print(p)
            # Cria uma página de cabeçalho com o número do contrato
            buffer = io.BytesIO()
            c = canvas.Canvas(buffer, pagesize=letter)
            c.setFont("Helvetica-Bold", 14)
            width, height = letter
            texto = f"Contrato Número: {p}"
            print(texto)
            c.drawCentredString(width / 2, height / 2, texto)
            c.showPage()
            c.save()
            buffer.seek(0)

            leitor_temp = PyPDF2.PdfReader(buffer)
            for pagina_temp in leitor_temp.pages:
                escritor_final.add_page(pagina_temp)

            # Determina o caminho completo do PDF a ser mesclado
            if os.path.exists(arquivo):
                caminho_completo = arquivo
            else:
                caminho_completo = os.path.join(pasta_final, arquivo)

            with open(caminho_completo, "rb") as pdf_in:
                leitor = PyPDF2.PdfReader(pdf_in)
                for pagina in leitor.pages:
                    escritor_final.add_page(pagina)

        nome_pdf_final = os.path.join(pasta_final, f"{anexo}_{lote}_parte_{i}.pdf")
        with open(nome_pdf_final, "wb") as saida:
            escritor_final.write(saida)
        final_pdfs.append(nome_pdf_final)

    # Agrupa os PDFs finais em subpastas: cada subpasta terá, no máximo, 14 documentos e tamanho total <= 140 MB.
    # Cria um mapeamento de cada PDF final para a sua subpasta.
    pdf_to_subfolder = {}
    max_docs = 14
    max_folder_size = 140 * 1024 * 1024  # 140 MB em bytes
    current_group_files = []
    current_group_size = 0
    folder_index = contador_inicial

    for pdf_file in final_pdfs:
        file_size = os.path.getsize(pdf_file)
        if (len(current_group_files) >= max_docs) or (current_group_size + file_size > max_folder_size):
            subfolder = os.path.join(pasta_final, f"{folder_index}_{lote}_grupo")
            os.makedirs(subfolder, exist_ok=True)
            for f_pdf in current_group_files:
                destino = os.path.join(subfolder, os.path.basename(f_pdf))
                os.rename(f_pdf, destino)
                pdf_to_subfolder[os.path.basename(f_pdf)] = f"{folder_index}_{lote}_grupo"
            folder_index += 1
            current_group_files = []
            current_group_size = 0
        current_group_files.append(pdf_file)
        current_group_size += file_size

    if current_group_files:
        subfolder = os.path.join(pasta_final, f"{folder_index}_{lote}_grupo")
        os.makedirs(subfolder, exist_ok=True)
        for f_pdf in current_group_files:
            destino = os.path.join(subfolder, os.path.basename(f_pdf))
            os.rename(f_pdf, destino)
            pdf_to_subfolder[os.path.basename(f_pdf)] = f"{folder_index}_{lote}_grupo"

    # Cria o arquivo Excel consolidado com os dados dos contratos e lotes,
    # incluindo a coluna "Pasta Interna" que indica a subpasta onde o PDF foi salvo.
    df = pd.DataFrame(dados_excel)
    df["Pasta Interna"] = df["Lote"].apply(lambda lote: pdf_to_subfolder.get(lote, ""))
    df.to_excel(os.path.join(str(caminho_base) + f"_{lote}_partes", f"{anexo}_{lote}_partes.xlsx"), index=False)

    # Remove arquivos temporários
    temp_files = ["temp_part.pdf"]
    for temp_file in temp_files:
        if os.path.exists(temp_file):
            os.remove(temp_file)

    # Compacta as pastas criadas em arquivos zip
    for p in glob.glob(str(caminho_base) + f"_{lote}_partes/*.pdf"):
        os.remove

    shutil.make_archive(str(caminho_base) + f"_{lote}_partes", 'zip', str(caminho_base) + f"_{lote}_partes")
    shutil.rmtree(str(caminho_base) + f"_{lote}_partes")
    return folder_index+1

## Processamento

In [81]:
tipos_colunas = {'n° do contrato': str, 'Razão Social_Nome Completo do Arrendatário': str, 'CNPJ_CPF do Arrendatário': str, 'Nº de Parcelas Contratadas': str, 'Valor do Bem': float}
data_columns = ['data do contrato', "Liquidação"]

df_base = pd.read_excel(path_base, dtype=tipos_colunas)
df_base['N° de parcelas contratadas'] = pd.to_numeric(df_base['N° de parcelas contratadas'], errors='coerce').fillna(0).astype(int)

df_base['CNPJ_CPF do Arrendatário'] = df_base['CNPJ_CPF do Arrendatário'].apply(formatar_cpf_cnpj)

df_base['Razão Social_Nome Completo do Arrendatário'] = df_base['Razão Social_Nome Completo do Arrendatário'].apply(to_pascal_case)

# Exemplo de uso
df_base = format_date_columns(df_base, data_columns)
df_base['n° do contrato'] = df_base['n° do contrato'].astype(str)  

In [82]:
df_base = df_base[["n° do contrato","Razão Social_Nome Completo do Arrendatário","CNPJ_CPF do Arrendatário","Valor do Bem","data do contrato","Liquidação","N° de parcelas contratadas"]]
df_base.columns =[
"Nº do Contrato",
"Cliente",
"CNPJ/CPF",
"Valor do Bem",
"Data do Contrato",
"Data de Liquidação",
"Quantidade de Parcelas"
]


In [83]:

import threading
import traceback
import sys
from tqdm import tqdm

def safe_criar_folha_rosto(contrato, info, comparativo, caminho_destino,
                           df_contratuais_vs_registro_contabil,
                           df_demonstracao_valor_anual,
                           df_efeito_resultado_contabil,
                           df_base_calculo_irpj,
                           df_base_calculo_csll,
                           df_demonstrativo_efeito_nulo,
                           caminho_base,
                           replace):
    try:
        criar_folha_rosto(contrato,
                          info,
                          comparativo,
                          caminho_destino,
                          df_contratuais_vs_registro_contabil,
                          df_demonstracao_valor_anual,
                          df_efeito_resultado_contabil,
                          df_base_calculo_irpj,
                          df_base_calculo_csll,
                          df_demonstrativo_efeito_nulo,
                          caminho_base,
                          replace)
    except Exception as e:
        # imprime nome do grupo, mensagem de erro e traceback completo
        print(f"[ERRO] no grupo {contrato}: {e}", file=sys.stderr)
        #traceback.print_exc()

# coisa

In [84]:
contratos_validos = comparativo["Contrato"].values.tolist()



In [ ]:

df_base = df_base[df_base["Nº do Contrato"].isin(contratos_validos)]
info_contratos  = df_base.groupby('Nº do Contrato')
replace = True


In [86]:

warnings.filterwarnings("ignore")
thread_limit=30
idx=0
threads = []

for contrato, info in tqdm(info_contratos, desc="Processando grupos", total=len(info_contratos)):
    thread = threading.Thread(
        target=safe_criar_folha_rosto,
        args=(contrato,
              info,
              comparativo,
              output_excel,
              df_contratuais_vs_registro_contabil,
              df_demonstracao_valor_anual,
              df_efeito_resultado_contabil,
              df_base_calculo_irpj,
              df_base_calculo_csll,
              df_demonstrativo_efeito_nulo,
              caminho_base,
              replace)
    )
    thread.start()
    threads.append(thread)

    if len(threads)>=thread_limit or idx==len(info_contratos)-1:
        # Aguarde até que todas as threads terminem
        for thread in threads:
            thread.join()
        threads =[]
    idx+=1


    

Processando grupos:   0%|          | 0/150 [00:00<?, ?it/s]

Processando grupos: 100%|██████████| 150/150 [01:52<00:00,  1.33it/s]
